In [6]:
# =============================================================================
# SETUP
# =============================================================================
import pandas as pd
import polars as pl
from football_pipeline.utils.constants import get_data_path

# Read the competitions data
bronze_events = pl.read_parquet(get_data_path("bronze", "open_data", "events"))
event_files = list(get_data_path("bronze", "open_data", "events").glob("*.parquet"))
print(f"Found {len(event_files)} event files")
print("Sample files:", [f.name for f in event_files[:5]])

SchemaError: extra column in file outside of expected schema: pass_outswinging, hint: specify this column in the schema, or pass extra_columns='ignore' in scan options